<a href="https://colab.research.google.com/github/Mye0n9/AAI_Pitcher/blob/CNN_Testing/Pitcher_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
import glob
import cv2
import re
import random
from google.colab.patches import cv2_imshow


print("NumPy Version :{}".format(np.__version__))
print("TensorFlow Version :{}".format(tf.__version__))
print("Matplotlib Version :{}".format(plt.matplotlib.__version__))

NumPy Version :1.22.4
TensorFlow Version :2.12.0
Matplotlib Version :3.7.1


# **DATA_PREPROCESSING**

In [2]:
image_data = []
label_data = []
for i in range(1,9):
  for j in range(1,33):
    image_link = '/content/drive/MyDrive/Raw_Data/{one}/{two}/*'.format(one = i,two = j)
    label_link = '/content/drive/MyDrive/Label/{one}/{two}/*'.format(one = i,two = j)
    image_data.append(glob.glob(image_link))
    label_data.append(glob.glob(label_link))


In [3]:
def cropper (img, point, num1, num2, size):
  # num1 is x
  # num2 is y
  max_x = 1920
  max_y = 1080

  x,y = point[num1], point[num2]
  x1,x2,y1,y2 = 0,0,0,0

  # x1 is smaller than x2
  # y1 is smaller than y1

  if point[num1]+size > max_x :
    # print('1',end='')
    x1 = point[num1] - size*2
    x2 = point[num1]
  else:
    # print('2',end='')
    x1 = point[num1]-size
    x2 = point[num1]+size
  
  if point[num2] + size > max_y:
    # print('3')
    y1 = point[num2] - size*2
    y2 = point[num2]
  else:
    # print('4')
    y1 = point[num2] - size
    y2 = point[num2] + size

  cropped_img = img[y1:y2, x1:x2]
  # print(cropped_img)
  return cropped_img

In [4]:
import json # 나중에 올리기
shoulders = []
elbows = []
wrists = []
IMG_SIZE = 64

# testing only one
# to test every data change 1 to len(image_data)
for i in range(1):
  for j in range(len(image_data[i])):
    f = open(label_data[i][j])
    img = cv2.imread(image_data[i][j],cv2.IMREAD_COLOR)
    data = json.load(f)
    points = data['annotations'][1]['points']
    # shoulder (6,7 -> 15~20)
    shoulder_point = points[15:21]
    # print(shoulder_point)
    shoulders.append(cropper(img,shoulder_point,0,1,int(IMG_SIZE/2)))
    shoulders.append(cropper(img,shoulder_point,3,4,int(IMG_SIZE/2)))
    # elbows (8,9 -> 21~27)
    elbow_point = points[21:27]
    elbows.append(cropper(img,elbow_point,0,1,int(IMG_SIZE/2)))
    elbows.append(cropper(img,elbow_point,3,4,int(IMG_SIZE/2)))
    # wrists (10,11 -> 27~33)
    wrist_point = points[27:33]
    wrists.append(cropper(img,wrist_point,0,1,int(IMG_SIZE/2)))
    wrists.append(cropper(img,wrist_point,3,4,int(IMG_SIZE/2)))
    f.close()


# Model

In [5]:
from sklearn.model_selection import train_test_split

x_Data = shoulders + elbows+ wrists

y_Data = [0 for _ in range(len(shoulders))]+ [1 for _ in range(len(elbows))] + [2 for _ in range(len(wrists))]
# y_Data = ['shoulder' for _ in range(len(shoulders))]+ ['elbow' for _ in range(len(elbows))] + ['wrist' for _ in range(len(wrists))]

x_train, x_test, y_train, y_test = train_test_split(x_Data, y_Data, test_size=0.2, random_state=42)

In [6]:
x_train = np.array(x_train)/255
x_test = np.array(x_test)/255
y_train = np.array(y_train)/255
y_test = np.array(y_test)/255

In [7]:
n_class = 3
model = tf.keras.models.Sequential()
# conv 1
model.add(tf.keras.layers.Conv2D(64,3,padding='same',activation='relu',input_shape=(IMG_SIZE,IMG_SIZE,3)))
model.add(tf.keras.layers.Dropout(rate=0.5))                          #  DO1
model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
# conv 2
model.add(tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'))
model.add(tf.keras.layers.Dropout(rate=0.5))                          # DO2
model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
# conv 3
model.add(tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'))
model.add(tf.keras.layers.Dropout(rate=0.5))                           # DO3
model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# dense layers
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(n_class, activation='softmax')) 

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 64, 64)        1792      
                                                                 
 dropout (Dropout)           (None, 64, 64, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 64)        36928     
                                                                 
 dropout_1 (Dropout)         (None, 32, 32, 64)        0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 16, 16, 64)       0         
 2D)                                                    

In [8]:
model.compile(optimizer='SGD',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [9]:
saved_model = '../files/pitcher_cnn3_model.h5'
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5,restore_best_weights=True)
mc = tf.keras.callbacks.ModelCheckpoint(saved_model, monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

In [10]:
history = model.fit(x_train, y_train, epochs=100,
                      callbacks=[es,mc],
                      validation_data=(x_test, y_test)
                 )

Epoch 1/100
15/15 [==============================] - ETA: 0s - loss: 0.0530 - accuracy: 0.3193
Epoch 1: val_accuracy improved from -inf to 0.38053, saving model to ../files/pitcher_cnn3_model.h5
15/15 [==============================] - 12s 50ms/step - loss: 0.0530 - accuracy: 0.3193 - val_loss: 0.3310 - val_accuracy: 0.3805
Epoch 2/100
11/15 [=====================>........] - ETA: 0s - loss: 0.0013 - accuracy: 0.3125
Epoch 2: val_accuracy did not improve from 0.38053
15/15 [==============================] - 0s 13ms/step - loss: 0.0012 - accuracy: 0.3215 - val_loss: 0.3053 - val_accuracy: 0.3805
Epoch 3/100
11/15 [=====================>........] - ETA: 0s - loss: 9.3016e-04 - accuracy: 0.3324
Epoch 3: val_accuracy did not improve from 0.38053
15/15 [==============================] - 0s 14ms/step - loss: 8.4612e-04 - accuracy: 0.3215 - val_loss: 0.2875 - val_accuracy: 0.3805
Epoch 4/100
11/15 [=====================>........] - ETA: 0s - loss: 6.3903e-04 - accuracy: 0.3068
Epoch 4: val_ac